# weather model
- 0 : 정상
- 1 : 눈
- 2 : 비

In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 

/data/home/ubuntu/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'HEIGHT':224,
    'WIDTH':224,
    'EPOCHS':2,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':16,
    'SEED':41
}


In [15]:
df = pd.read_csv('./data/train_new_label.csv')
df

,sample_id,video_path,label,crash_ego,weather,timing
0,TRAIN_0000,./train/TRAIN_0000.mp4,7,2,0,0
1,TRAIN_0001,./train/TRAIN_0001.mp4,7,2,0,0
2,TRAIN_0002,./train/TRAIN_0002.mp4,0,0,Na,Na
3,TRAIN_0003,./train/TRAIN_0003.mp4,0,0,Na,Na
4,TRAIN_0004,./train/TRAIN_0004.mp4,1,1,0,0
...,...,...,...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3,1,1,0
2694,TRAIN_2694,./train/TRAIN_2694.mp4,5,1,2,0
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0,0,Na,Na
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0,0,Na,Na


In [16]:
df = df[df['label']!=0].reset_index(drop=True)
df

,sample_id,video_path,label,crash_ego,weather,timing
0,TRAIN_0000,./train/TRAIN_0000.mp4,7,2,0,0
1,TRAIN_0001,./train/TRAIN_0001.mp4,7,2,0,0
2,TRAIN_0004,./train/TRAIN_0004.mp4,1,1,0,0
3,TRAIN_0006,./train/TRAIN_0006.mp4,3,1,1,0
4,TRAIN_0007,./train/TRAIN_0007.mp4,7,2,0,0
...,...,...,...,...,...,...
910,TRAIN_2685,./train/TRAIN_2685.mp4,8,2,0,1
911,TRAIN_2689,./train/TRAIN_2689.mp4,1,1,0,0
912,TRAIN_2692,./train/TRAIN_2692.mp4,7,2,0,0
913,TRAIN_2693,./train/TRAIN_2693.mp4,3,1,1,0


In [17]:
df['new_weather'] = ''
for i in range(len(df)):
    if ((df.loc[i, 'weather'] == '0') & (df.loc[i, 'timing'] == '0')):
        df.loc[i, 'new_weather'] = 0
        
    elif ((df.loc[i, 'weather'] == '0') & (df.loc[i, 'timing'] == '1')):
        df.loc[i, 'new_weather'] = 1
    
    elif ((df.loc[i, 'weather'] == '1') & (df.loc[i, 'timing'] == '0')):
        df.loc[i, 'new_weather'] = 2
    
    elif ((df.loc[i, 'weather'] == '1') & (df.loc[i, 'timing'] == '1')):
        df.loc[i, 'new_weather'] = 3
        
    elif ((df.loc[i, 'weather'] == '2') & (df.loc[i, 'timing'] == '0')):
        df.loc[i, 'new_weather'] = 4
        
    elif ((df.loc[i, 'weather'] == '2') & (df.loc[i, 'timing'] == '1')):
        df.loc[i, 'new_weather'] = 5
            
df 

,sample_id,video_path,label,crash_ego,weather,timing,new_weather
0,TRAIN_0000,./train/TRAIN_0000.mp4,7,2,0,0,0
1,TRAIN_0001,./train/TRAIN_0001.mp4,7,2,0,0,0
2,TRAIN_0004,./train/TRAIN_0004.mp4,1,1,0,0,0
3,TRAIN_0006,./train/TRAIN_0006.mp4,3,1,1,0,2
4,TRAIN_0007,./train/TRAIN_0007.mp4,7,2,0,0,0
...,...,...,...,...,...,...,...
910,TRAIN_2685,./train/TRAIN_2685.mp4,8,2,0,1,1
911,TRAIN_2689,./train/TRAIN_2689.mp4,1,1,0,0,0
912,TRAIN_2692,./train/TRAIN_2692.mp4,7,2,0,0,0
913,TRAIN_2693,./train/TRAIN_2693.mp4,3,1,1,0,2


In [18]:
df.new_weather.value_counts()

0    635
2    112
1     81
4     61
3     17
5      9
Name: new_weather, dtype: int64

In [ ]:
df = df.drop(columns=['label','crash_ego','weather','timing'])
df = df.rename(columns={'new_weather' : 'weather'})
df

In [20]:
df.to_csv('./data/train_weather_timing.csv',index=False)

In [28]:
df = pd.read_csv('./data/train_weather_timing.csv')
df.head()

,sample_id,video_path,weather
0,TRAIN_0000,/data/home/ubuntu/workspace/dacon/data/train/T...,0
1,TRAIN_0001,/data/home/ubuntu/workspace/dacon/data/train/T...,0
2,TRAIN_0004,/data/home/ubuntu/workspace/dacon/data/train/T...,0
3,TRAIN_0006,/data/home/ubuntu/workspace/dacon/data/train/T...,2
4,TRAIN_0007,/data/home/ubuntu/workspace/dacon/data/train/T...,0


In [29]:
# 초반 30프레임만 사용
# 0- 15프레임마다 1개씩 -> 635*2 = 1270
# 1- 2프레임마다 1개씩 -> 81*15 = 1215
# 2- 3프레임마다 1개씩 -> 112*10 = 1120
# 3- 1프레임마다 1개씩 -> 17*30 = 510
# 4- 2프레임마다 1개씩 -> 61*30 = 915
# 5- 1프레임마다 1개씩 -> 9*30 = 270

In [32]:
def get_img(path, label=False):
    print(path)
    print(label)
    frames, labels = [], []
    cap = cv2.VideoCapture(path)
    cnt = 0

    if (label=='0'):
        divide = 15
    elif (label=='1'):
        divide = 2
    elif (label=='2'):
        divide = 3
    elif (label=='3'):
        divide = 1
    elif (label=='4'):
        divide = 2
    elif (label=='5'):
        divide = 1                        
        
    if (label):
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            cnt+=1
            if (cnt%divide==0):
                img = cv2.resize(img, (CFG['HEIGHT'], CFG['WIDTH']))
                img = img / 255.
                frames.append(img)
                labels.append(int(label))
            if (cnt==30):
                break
        return frames, labels
                
    else:
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            cnt+=1
            if (cnt%3==0):
                img = cv2.resize(img, (CFG['HEIGHT'], CFG['WIDTH']))
                img = img / 255.
                frames.append(img)
            if (cnt==30):
                break
        return frames

In [33]:
frames, labels = get_img(df.loc[0,'video_path'], df.loc[0,'weather'])

/data/home/ubuntu/workspace/dacon/data/train/TRAIN_0000.mp4
0


ValueError: too many values to unpack (expected 2)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sample_id   915 non-null    object
 1   video_path  915 non-null    object
 2   weather     915 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.6+ KB


In [34]:
df.loc[0,'weather']

0

In [3]:
df = pd.read_csv('./data/train_weather.csv')
df['weather'].value_counts()

Na    1783
0      716
1      129
2       70
Name: weather, dtype: int64

In [ ]:
# 0은 10프레임마다 1개씩 -> 716*3 = 2148
# 1은 2프레임마다 1개씩 -> 129*15 = 1935
# 2는 1프레임마다 1개씩 -> 70*30 = 2100

In [46]:
df = pd.read_csv('./data/train_weather.csv')
df = df[df['weather']!='Na'].reset_index(drop=True)
# df['snow'] = df['weather'].apply(lambda x: 0 if x!='1' else 1)
df['snow'] = ''
df['rain'] = ''
for i in range(len(df)):
    if(df.loc[i, 'weather'] == '0'):
       df.loc[i, 'snow'] = 0
       df.loc[i, 'rain'] = 0
       
    elif(df.loc[i, 'weather'] == '1'):
        df.loc[i, 'snow'] = 1
        df.loc[i, 'rain'] = None
        
    elif(df.loc[i, 'weather'] == '2'):
        df.loc[i, 'snow'] = None
        df.loc[i, 'rain'] = 1

df_snow = df.drop(columns=['weather', 'rain']).dropna().reset_index(drop=True)
df_rain = df.drop(columns=['weather', 'snow']).dropna().reset_index(drop=True)

In [45]:
df_snow.snow.value_counts()

0    716
1    129
Name: snow, dtype: int64

In [50]:
df_snow.to_csv('./data/train_weather_snow.csv', index=False)

In [ ]:
# 716 * 5 = 3580 (6프레임마다)
# 129 * 30 = 3870

In [49]:
df_rain.rain.value_counts()

0    716
1     70
Name: rain, dtype: int64

In [ ]:
# 716 * 3 = 2148 (10프레임마다)
# 70 * 30 = 2100

In [51]:
df_rain.to_csv('./data/train_weather_rain.csv', index=False)

In [33]:
ori = pd.read_csv('./data/test_weather_230309.csv')
new = pd.read_csv('./data/test_weather_230310.csv')
new.rename(columns={'weather' : 'new_weather'}, inplace=True)
compare = ori.merge(new).drop(columns='video_path')
compare

,sample_id,weather,new_weather
0,TEST_0000,0,0
1,TEST_0001,0,2
2,TEST_0002,0,0
3,TEST_0003,0,0
4,TEST_0004,0,0
...,...,...,...
1795,TEST_1795,0,0
1796,TEST_1796,0,0
1797,TEST_1797,0,1
1798,TEST_1798,0,0


In [36]:
from sklearn.metrics import f1_score
f1_score(compare['weather'], compare['new_weather'], average=None)

array([0.89608287, 0.52631579, 0.14693878])

In [12]:
def get_img(path, label=False):
    frames, labels = [], []
    cap = cv2.VideoCapture(path)
    cnt = 0
    # normal 라벨 기준 몇배 차이나는지 확인{0: 10, 1: 2, 2: 1}
    if (label=='0'):
        divide = 10
    elif (label=='1'):
        divide = 2
    elif (label=='2'):
        divide = 1
        
    if (label):
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            cnt+=1
            if (cnt%divide==0):
                img = cv2.resize(img, (CFG['HEIGHT'], CFG['WIDTH']))
                img = img / 255.
                frames.append(img)
                labels.append(int(label))
        return frames, labels
                
    else:
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            cnt+=1
            if (cnt%5==0):
                img = cv2.resize(img, (CFG['HEIGHT'], CFG['WIDTH']))
                img = img / 255.
                frames.append(img)   
        return frames

In [13]:
class CustomDataset(Dataset):
    def __init__(self, frames, labels):
        self.frames = frames
        self.labels = labels

        
    def __getitem__(self, index):
        frame = self.transform_frame(self.frames[index])
        if self.labels is not None:
            label = self.labels[index]
            return frame, label
        else:
            return frame
        
        
    def __len__(self):
        return len(self.frames)
    
    def transform_frame(self, frame):
        frame = frame / 255.
        return torch.FloatTensor(np.array(frame)).permute(2, 0, 1)

In [14]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

In [15]:
## 모델 저장하는 부분 추가
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        torch.save(model.state_dict(), './car_crash_r3d_{0:02d}.ckpt'.format(epoch))
        print(f'======== model saved - epoch : ', epoch)

        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [16]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [17]:
ckp = torch.load('D:/ㅎㅎㄱ/0.Study/dacon_230221/ckp/weather_res101_36.ckpt')

In [18]:
model = models.resnet101()

num_classes = 3
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

device = torch.device('cuda:0')
model.load_state_dict(ckp)
model = model.to(device)

# test = pd.read_csv('./data/test.csv')

In [19]:
test = pd.read_csv('./data/test.csv')

In [20]:
total_frames = []
for i in tqdm(range(len(test))):
    frames = get_img(test.loc[i,'video_path'], None)
    total_frames.extend(frames)

  0%|          | 0/1800 [00:00<?, ?it/s]

In [62]:
test_dataset = CustomDataset(total_frames, None)
test_loader = DataLoader(
            test_dataset, 
            batch_size = CFG['BATCH_SIZE'],
            shuffle=False, 
            num_workers=0
            )

In [66]:
from scipy.stats import mode
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    img_preds = []
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device) 
            logit = model(videos)
            img_preds += logit.argmax(1).detach().cpu().numpy().tolist()
    
    
    for i in range(0, len(img_preds), 10):
        preds.append(int(mode(img_preds[i: i+10]).mode))

    return preds

In [67]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
preds = inference(model, test_loader, device)
test['weather'] = preds
test.to_csv('./data/test_weather.csv', index=False)

  0%|          | 0/1125 [00:00<?, ?it/s]

In [69]:
test

,sample_id,video_path,weather
0,TEST_0000,D:/ㅎㅎㄱ/0.Study/dacon_230221/test/TEST_0000.mp4,0
1,TEST_0001,D:/ㅎㅎㄱ/0.Study/dacon_230221/test/TEST_0001.mp4,0
2,TEST_0002,D:/ㅎㅎㄱ/0.Study/dacon_230221/test/TEST_0002.mp4,0
3,TEST_0003,D:/ㅎㅎㄱ/0.Study/dacon_230221/test/TEST_0003.mp4,0
4,TEST_0004,D:/ㅎㅎㄱ/0.Study/dacon_230221/test/TEST_0004.mp4,0
...,...,...,...
1795,TEST_1795,D:/ㅎㅎㄱ/0.Study/dacon_230221/test/TEST_1795.mp4,0
1796,TEST_1796,D:/ㅎㅎㄱ/0.Study/dacon_230221/test/TEST_1796.mp4,0
1797,TEST_1797,D:/ㅎㅎㄱ/0.Study/dacon_230221/test/TEST_1797.mp4,0
1798,TEST_1798,D:/ㅎㅎㄱ/0.Study/dacon_230221/test/TEST_1798.mp4,0


In [11]:
df = pd.read_csv('./data/train_weather.csv')
df = df[df['weather']!='Na'].reset_index(drop=True)
df

,sample_id,video_path,weather
0,TRAIN_0000,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0000.mp4,0
1,TRAIN_0001,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0001.mp4,0
2,TRAIN_0004,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0004.mp4,0
3,TRAIN_0006,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0006.mp4,1
4,TRAIN_0007,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0007.mp4,0
...,...,...,...
910,TRAIN_2685,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_2685.mp4,0
911,TRAIN_2689,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_2689.mp4,0
912,TRAIN_2692,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_2692.mp4,0
913,TRAIN_2693,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_2693.mp4,1


In [12]:
df = df[:10]
df

,sample_id,video_path,weather
0,TRAIN_0000,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0000.mp4,0
1,TRAIN_0001,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0001.mp4,0
2,TRAIN_0004,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0004.mp4,0
3,TRAIN_0006,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0006.mp4,1
4,TRAIN_0007,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0007.mp4,0
5,TRAIN_0009,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0009.mp4,1
6,TRAIN_0011,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0011.mp4,0
7,TRAIN_0015,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0015.mp4,0
8,TRAIN_0016,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0016.mp4,0
9,TRAIN_0017,D:/ㅎㅎㄱ/0.Study/dacon_230221/train/TRAIN_0017.mp4,0


In [13]:
seed_everything(CFG['SEED']) # Seed 고정

In [14]:
total_frames, total_labels = [], []
for i in tqdm(range(len(df))):
    frames, labels = get_img(df.loc[i,'video_path'], df.loc[i,'weather'])
    total_frames.extend(frames)
    total_labels.extend(labels)

  0%|          | 0/10 [00:00<?, ?it/s]

In [15]:
train_set, val_set, train_label, val_lable = train_test_split(total_frames, total_labels, test_size=0.2, random_state=CFG['SEED'])

In [16]:
train_dataset = CustomDataset(train_set, train_label)
train_loader = DataLoader(
    train_dataset, 
    batch_size = CFG['BATCH_SIZE'],
    shuffle=True, 
    num_workers=0
    )

val_dataset = CustomDataset(val_set, val_lable)
val_loader = DataLoader(
    val_dataset, 
    batch_size = CFG['BATCH_SIZE'],
    shuffle=False, 
    num_workers=0
    )

In [24]:
# resnet50
model = models.resnet101()

num_classes = 3
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

device = torch.device('cuda:0')
model = model.to(device)

optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='max', 
    factor=0.5, 
    patience=2,
    threshold_mode='abs',
    min_lr=1e-4, 
    verbose=False
)
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.32783] Val Loss : [6.81932] Val F1 : [0.28000]
======== model saved - epoch :  1


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.12133] Val Loss : [0.03099] Val F1 : [1.00000]
======== model saved - epoch :  9


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.13523] Val Loss : [0.24900] Val F1 : [0.80364]
======== model saved - epoch :  10
